In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Blr_data.csv')

In [3]:
df.drop(['area_type','availability','society','balcony','balcony'],axis=1,inplace=True)

In [4]:
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [5]:
import string
alphabet_l = string.ascii_lowercase
def rem_str(x):
    if str(x).find('-') != -1:
        les = str(x).split('-')
        avg = (float(les[0]) + float(les[1]))/2
        return avg
    for alpha in alphabet_l:
        if alpha in x:
            return 0
    return x
    

In [6]:
df.total_sqft = df.total_sqft.apply(rem_str)
df = df[df.total_sqft != 0]

In [7]:
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [8]:
df['size'].fillna('2 BHK')
df['size'] = df['size'].apply(lambda x:str(x).split(' ')[0])


In [9]:
df.bath = df.bath.fillna(df.bath.median())

In [10]:
df.dropna(inplace=True)

In [11]:
locations  =  df.location.value_counts()

In [12]:
locations_less_10 = locations[locations<=10]

In [13]:
len(locations_less_10)

1059

In [14]:
df.location = df.location.apply(lambda x:x if x not in locations_less_10 else 'other')

In [15]:
df.location.value_counts()

other                        2891
Whitefield                    538
Sarjapur  Road                399
Electronic City               302
Kanakpura Road                271
                             ... 
Doddaballapur                  11
2nd Phase Judicial Layout      11
Nehru Nagar                    11
LB Shastri Nagar               11
Marsur                         11
Name: location, Length: 241, dtype: int64

In [16]:
len(locations_less_10)

1059

In [17]:
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2,1056,2.0,39.07
1,Chikka Tirupathi,4,2600,5.0,120.00
2,Uttarahalli,3,1440,2.0,62.00
3,Lingadheeranahalli,3,1521,3.0,95.00
4,Kothanur,2,1200,2.0,51.00


In [18]:
df['total_sqft'] = df['total_sqft'].apply(lambda x:float(x))
df['price_per_sqft'] = df['price']*100000 / df['total_sqft']

In [19]:
df.head()

,location,size,total_sqft,bath,price,price_per_sqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000


In [20]:
df['size'] = df['size'].apply(lambda x:float(x))
df = df[df['total_sqft']/df['size'] >=300]

In [21]:
df.head()

,location,size,total_sqft,bath,price,price_per_sqft
0,Electronic City Phase II,2.0,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4.0,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3.0,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3.0,1521.0,3.0,95.00,6245.890861
4,Kothanur,2.0,1200.0,2.0,51.00,4250.000000


In [22]:
# df.price_per_sqft.mean()
df.price_per_sqft.std()

4160.931609514924

In [23]:
df.price_per_sqft.describe()

count     12513.000000
mean       6307.441644
std        4160.931610
min         267.829813
25%        4210.526316
50%        5295.007564
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

In [24]:
def rem_price_outlier(df):
    new_df = pd.DataFrame()
    for key,sub_df in df.groupby('location'):
        m = sub_df.price_per_sqft.mean()
        std = sub_df.price_per_sqft.std()
        gen_df = sub_df[(sub_df.price_per_sqft > (m-std)) & (sub_df.price_per_sqft < (m+std))]
        new_df =  pd.concat([new_df,gen_df],ignore_index=True)
    return new_df

In [25]:
new_df = rem_price_outlier(df)

In [26]:
new_df.shape

(10340, 6)

In [27]:
df.shape

(12513, 6)

In [28]:
import numpy as np
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk,bhk_df in location_df.groupby('size'):
            bhk_stats[bhk] = {
                'mean' : bhk_df.price_per_sqft.mean(),
                'std' :  bhk_df.price_per_sqft.std(),
                'count' : bhk_df.shape[0]
            }
            
        for bhk,bhk_df in location_df.groupby('size'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft > (stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
            

In [29]:
new_df =bhk_outlier_remover(new_df)

In [30]:
new_df.shape

(7027, 6)

In [31]:
new_df.drop(['price_per_sqft'],axis=1,inplace=True)

In [32]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv('cleaned_data.csv')

In [ ]:
X = new_df.drop(['price'],axis=1)
y = new_df.price

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
col_trans =make_column_transformer((OneHotEncoder(sparse=False),['location']),remainder='passthrough')

In [ ]:
scaler = StandardScaler()

In [ ]:
lr = LinearRegression()

In [ ]:
pipe = make_pipeline(col_trans,scaler,lr)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
import pickle
with open('pipe.pkl','wb') as f:
    pickle.dump(pipe,f)
    

In [ ]:
X_train.head()

In [ ]:
pipe.predict([['other',3.0,2000.0,2.0]])